In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 패키지 임포트, 파일 불러오기
import pandas as pd

service = pd.read_csv('/content/drive/MyDrive/서울시 개인서비스 요금 정보.csv', encoding='cp949')
price = pd.read_csv('/content/drive/MyDrive/서울시 착한가격업소 현황.csv', encoding='cp949')
grocery = pd.read_csv('/content/drive/MyDrive/생필품 농수축산물 가격 정보(2023년).csv', encoding='cp949')

In [ ]:
# 면적을 데이터 값이 106개임. 나머지는 결측치이므로 drop
service.info()
service.drop(columns=['면적(㎡)'], inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8630 entries, 0 to 8629
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   업소일련번호  8630 non-null   int64  
 1   업소명     8630 non-null   object 
 2   동명      8630 non-null   object 
 3   주소      8628 non-null   object 
 4   면적(㎡)   106 non-null    float64
 5   전화번호    8246 non-null   object 
 6   업종      8630 non-null   object 
 7   품목코드    8630 non-null   int64  
 8   품목      8630 non-null   object 
 9   가격(원)   8600 non-null   float64
 10  점검일자    8630 non-null   object 
 11  구명      8630 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 809.2+ KB


In [ ]:
# 가격에 na값인 것을 찾아서 결측치를 보간해줍니다.
# 결측치는 30개이고 화양동, 자양동, 구의동, 중곡동(이상 광진구), 잠실동(송파)에 해당하는 곳들이어서 동을 기준으로 평균 값을 채워줍니다.
temp = service[service['가격(원)'].isna()]
temp
service.fillna({'가격(원)':service.groupby(['동명','품목'])['가격(원)'].transform('mean')}, inplace=True)

In [ ]:
# 구 + 동 별 품목의 가격을 알아봅니다.
price_by_gu = service.groupby(['구명','동명','업종','품목'])['가격(원)'].mean().reset_index()
price_by_gu.rename(columns={'가격(원)':'동 별 평균 가격'}, inplace=True)

In [ ]:
# 구 별 가격을 새로 만들어줍니다.
temp = service.groupby(['구명','업종','품목'])['가격(원)'].mean().reset_index()
price_by_gu = pd.merge(price_by_gu, temp, left_on = ['구명','업종','품목'], right_on = ['구명','업종','품목'], how='left')
price_by_gu.rename(columns={'가격(원)':'구 별 평균 가격'}, inplace=True)

In [ ]:
# service에 동 별 평균 가격, 구 별 평균 가격을 추가해줍니다.
service = pd.merge(service, price_by_gu, left_on = ['구명','동명','업종','품목'], right_on = ['구명','동명','업종','품목'], how='left')
service.drop(columns=['업소일련번호','품목코드'], inplace=True)

In [ ]:
# #GIS 위치를 찍어줍니다.

# # 라이브러리 설치
# !pip install geopy

# # import 라이브러리
# from geopy.geocoders import Nominatim
# from tqdm import tqdm

# #예제 데이터 : df_shake
# #컬럼 정보 : name, branch, addr

# # 위도, 경도 반환하는 함수
# def geocoding(address):
#     try:
#         geo_local = Nominatim(user_agent='South Korea')  #지역설정
#         location = geo_local.geocode(address)
#         geo = [location.latitude, location.longitude]
#         return geo

#     except:
#         return [0,0]

# # 실행
# for idx,addr in enumerate(tqdm(service.주소)):
#     service.loc[idx,'latitude'] = geocoding(addr)[0]
#     service.loc[idx,'longitude'] = geocoding(addr)[1]
# service.to_csv('service.csv')

In [ ]:
# 경도, 위도 반환 프로그램을 통해 경도, 위도를 반환시키고 파일을 다시 불러옵니다.
service = pd.read_csv('/content/drive/MyDrive/service_geo.csv', index_col=0)

In [ ]:
# 필요없는 열 제거
service.drop(columns=['CLSS','PNU','주소구분','표준신주소','우편번호','법정동코드','법정동명'], inplace=True)
service.dropna(inplace=True)

In [ ]:
# 태블로의 파일 연동 과정에서 geojson 파일 join을 위해 주소를 만들어줍니다.
service['시'] = service['입력주소'].str.split(' ').str[0]
service['geo_address'] = service['시'] + ' ' + service['구명'] + ' ' + service['행정동명']
service

,업소명,동명,주소,전화번호,업종,품목,가격(원),구명,동 별 평균 가격,구 별 평균 가격,입력주소,X,Y,표준구주소,행정동코드,행정동명,시,geo_address
0,가마로,화양동,서울특별시 광진구 동일로22길 100 (화양동),02-464-2202,한식,삼겹살,19400.00000,광진구,16840.000000,15953.663560,서울특별시 광진구 동일로22길 100 (화양동),127.069499,37.540867,서울특별시 광진구 화양동 7-42,1.121571e+09,화양동,서울특별시,서울특별시 광진구 화양동
1,모츠커피,화양동,"서울특별시 광진구 군자로3길 14 (화양동, 화양빌딩)",02-418-5507,다방업,커피(외식),3750.00000,광진구,3750.000000,3420.833333,"서울특별시 광진구 군자로3길 14 (화양동, 화양빌딩)",127.070448,37.544668,서울특별시 광진구 화양동 15-20,1.121571e+09,화양동,서울특별시,서울특별시 광진구 화양동
2,모츠커피,화양동,"서울특별시 광진구 군자로3길 14 (화양동, 화양빌딩)",02-418-5507,다방업,다방 국산차(녹차),6000.00000,광진구,4600.000000,4237.209302,"서울특별시 광진구 군자로3길 14 (화양동, 화양빌딩)",127.070448,37.544668,서울특별시 광진구 화양동 15-20,1.121571e+09,화양동,서울특별시,서울특별시 광진구 화양동
3,앤디샵179,화양동,서울특별시 광진구 능동로 179 (화양동),02-461-3370,미용업,미용료 (커트),22000.00000,광진구,20333.333330,15827.615780,서울특별시 광진구 능동로 179 (화양동),127.073513,37.546638,서울특별시 광진구 화양동 111-118,1.121571e+09,화양동,서울특별시,서울특별시 광진구 화양동
4,앤디샵179,화양동,서울특별시 광진구 능동로 179 (화양동),02-461-3370,미용업,미용료 (파마),61333.33333,광진구,61333.333330,47520.883290,서울특별시 광진구 능동로 179 (화양동),127.073513,37.546638,서울특별시 광진구 화양동 111-118,1.121571e+09,화양동,서울특별시,서울특별시 광진구 화양동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8625,죠이헤어클럽,쌍문동,서울특별시 도봉구 도봉로109길 26 (쌍문동),070-8940-4103,미용업,미용료 (커트),11000.00000,도봉구,14550.000000,13441.860470,서울특별시 도봉구 도봉로109길 26 (쌍문동),127.031914,37.647022,서울특별시 도봉구 쌍문동 100-32,1.132068e+09,쌍문3동,서울특별시,서울특별시 도봉구 쌍문3동
8626,옛집,쌍문동,"서울특별시 도봉구 도봉로109길 24 (쌍문동) ,성원펠리스101동103호",02-905-5509,한식,김치찌개 백반,8000.00000,도봉구,8166.666667,7933.333333,"서울특별시 도봉구 도봉로109길 24 (쌍문동) ,성원펠리스101동103호",127.032107,37.647015,서울특별시 도봉구 쌍문동 100-2,1.132068e+09,쌍문3동,서울특별시,서울특별시 도봉구 쌍문3동
8627,옛집,쌍문동,"서울특별시 도봉구 도봉로109길 24 (쌍문동) ,성원펠리스101동103호",02-905-5509,한식,삼겹살,15000.00000,도봉구,15000.000000,14545.454550,"서울특별시 도봉구 도봉로109길 24 (쌍문동) ,성원펠리스101동103호",127.032107,37.647015,서울특별시 도봉구 쌍문동 100-2,1.132068e+09,쌍문3동,서울특별시,서울특별시 도봉구 쌍문3동
8628,옛집,쌍문동,"서울특별시 도봉구 도봉로109길 24 (쌍문동) ,성원펠리스101동103호",02-905-5509,한식,돼지갈비(외식),13000.00000,도봉구,14000.000000,14333.333330,"서울특별시 도봉구 도봉로109길 24 (쌍문동) ,성원펠리스101동103호",127.032107,37.647015,서울특별시 도봉구 쌍문동 100-2,1.132068e+09,쌍문3동,서울특별시,서울특별시 도봉구 쌍문3동


In [ ]:
service.to_csv('service_geo.csv')

In [ ]:
# 착한 가게 데이터 확인
# 찾아오시는 길, 업소정보, 기준년월, 자랑거리는 시각화하는 데 필요하지 않음 -> 따라서 drop
price.info()
price.drop(columns=['찾아오시는 길','업소정보', '기준년월','자랑거리'], inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1867 entries, 0 to 1866
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   업소아이디    1867 non-null   int64 
 1   업소명      1867 non-null   object
 2   분류코드     1867 non-null   int64 
 3   분류코드명    1867 non-null   object
 4   업소 주소    1867 non-null   object
 5   업소 전화번호  1753 non-null   object
 6   찾아오시는 길  1040 non-null   object
 7   업소정보     1741 non-null   object
 8   자랑거리     1201 non-null   object
 9   추천수      1867 non-null   int64 
 10  기준년월     1867 non-null   object
 11  업소 사진    1867 non-null   object
dtypes: int64(3), object(9)
memory usage: 175.2+ KB


In [ ]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1867 entries, 0 to 1866
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   업소아이디    1867 non-null   int64 
 1   업소명      1867 non-null   object
 2   분류코드     1867 non-null   int64 
 3   분류코드명    1867 non-null   object
 4   업소 주소    1867 non-null   object
 5   업소 전화번호  1753 non-null   object
 6   추천수      1867 non-null   int64 
 7   업소 사진    1867 non-null   object
dtypes: int64(3), object(5)
memory usage: 116.8+ KB


In [ ]:
# 경도, 위도 반환 프로그램을 통해 경도, 위도를 반환시키고 파일을 다시 불러옵니다.
price.to_csv('price.csv')

In [ ]:
# 생필품 데이터셋 확인하기
grocery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76559 entries, 0 to 76558
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   일련번호               76559 non-null  int64  
 1   시장/마트 번호           76559 non-null  int64  
 2   시장/마트 이름           76559 non-null  object 
 3   품목 번호              76559 non-null  int64  
 4   품목 이름              76365 non-null  object 
 5   실판매규격              0 non-null      float64
 6   가격(원)              76559 non-null  int64  
 7   년도-월               76559 non-null  object 
 8   비고                 9995 non-null   object 
 9   시장유형 구분(시장/마트) 코드  76559 non-null  int64  
 10  시장유형 구분(시장/마트) 이름  76559 non-null  object 
 11  자치구 코드             76559 non-null  int64  
 12  자치구 이름             76559 non-null  object 
 13  점검일자               76559 non-null  object 
dtypes: float64(1), int64(6), object(7)
memory usage: 8.2+ MB


In [ ]:
# 실판매규격, 비고, 점검일자는 데이터를 시각화하는 데 필요없으므로 drop한다.
grocery['비고'].unique()
grocery.drop(columns=['실판매규격','비고','점검일자'], inplace=True)

In [ ]:
# 가격이 0원인 데이터가 말이 안되므로, 0인 것들을 찾아 평균 값으로 값을 대체한다.
grocery.loc[grocery['가격(원)'] == 0, '가격(원)'] = None
grocery['가격(원)'] = grocery.groupby(['시장/마트 이름','년도-월','품목 이름'])['가격(원)'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
# 제품 가격이 아예 안나오는 몇몇가지의 품목이 있으므로, 자치구 이름, '년도-월', 시장유형 구분, 품목 이름을 기준으로 그룹화하여 평균가격으로 가격을 대체시킨다.
grocery['가격(원)'] = grocery.groupby(['자치구 이름','년도-월','시장유형 구분(시장/마트) 이름', '품목 이름'])['가격(원)'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
# 그다음 처리되지 않은 nan값은 품목 이름과 년도-월을 기준으로 그룹화하여 평균 가격으로 대체하였으나, 그럼에도 값이 채워지지 않아, 품목의 평균 가격으로 대체하였다.
grocery['가격(원)'] = grocery.groupby(['품목 이름','년도-월'])['가격(원)'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
# 그다음 처리되지 않은 nan값은 품목 이름을 기준으로 그룹화하여 평균 가격으로 대체하였으나, 그럼에도 값이 채워지지 않아, 품목의 평균 가격으로 대체하였다.
grocery['가격(원)'] = grocery.groupby(['품목 이름'])['가격(원)'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
# 이외의 값은 그냥 drop한다.
grocery.dropna(inplace=True)

In [ ]:
# 데이터를 확인한다.
grocery.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76365 entries, 0 to 76558
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   일련번호               76365 non-null  int64  
 1   시장/마트 번호           76365 non-null  int64  
 2   시장/마트 이름           76365 non-null  object 
 3   품목 번호              76365 non-null  int64  
 4   품목 이름              76365 non-null  object 
 5   가격(원)              76365 non-null  float64
 6   년도-월               76365 non-null  object 
 7   시장유형 구분(시장/마트) 코드  76365 non-null  int64  
 8   시장유형 구분(시장/마트) 이름  76365 non-null  object 
 9   자치구 코드             76365 non-null  int64  
 10  자치구 이름             76365 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 7.0+ MB


In [ ]:
# grocery 데이터를 내보낸다.
grocery.to_csv('grocery_geo.csv')

In [ ]:
# price(착한가게업소) 데이터셋을 불러오고 시,구,동으로 나누기 위해 split한다.
price = pd.read_excel('/content/drive/MyDrive/price_geo.xlsx', index_col=0)
price['split'] = price['표준구주소'].str.split(' ')
price['시'] = price['split'].str[0]
price['구'] = price['split'].str[1]
price['동'] = price['split'].str[2]

In [ ]:
# 주소 생성하기
price['주소'] = price['시'] + ' ' + price['구'] + ' ' + price['행정동명']

In [ ]:
# 필요없는 열을 drop해준다.
price.drop(columns=['업소아이디','분류코드','표준신주소','행정동코드','CLSS', 'PNU', '행정동코드','행정동명','법정동코드','split','시'], inplace=True)

In [ ]:
price

,업소명,분류코드명,업소 주소,업소 전화번호,추천수,업소 사진,입력주소,X,Y,주소구분,표준구주소,우편번호,법정동명,구,동,주소
0,아폴로헤어크리닉,이미용,서울특별시 중랑구 사가정로50길 67 (면목동),495-7337,450,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 중랑구 사가정로50길 67 (면목동),127.085013,37.577850,새주소,서울특별시 중랑구 면목동 353-8,2240.0,면목동,중랑구,면목동,서울특별시 중랑구 면목7동
1,상록수미용실,이미용,서울특별시 중랑구 동일로99길 22 (면목동),NaN,432,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 중랑구 동일로99길 22 (면목동),127.078617,37.587919,새주소,서울특별시 중랑구 면목동 179-74,2135.0,면목동,중랑구,면목동,서울특별시 중랑구 면목2동
2,열린미용실,이미용,서울특별시 관악구 난곡로40길 16 (신림동),NaN,336,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 관악구 난곡로40길 16 (신림동),126.917674,37.475284,새주소,서울특별시 관악구 신림동 607-137,8849.0,신림동,관악구,신림동,서울특별시 관악구 미성동
3,미림17분칼라,기타서비스업,서울특별시 관악구 호암로26길 1 (신림동),02-871-7070,318,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 관악구 호암로26길 1 (신림동),126.933902,37.470989,새주소,서울특별시 관악구 신림동 1514-9,8812.0,신림동,관악구,신림동,서울특별시 관악구 대학동
4,덕성이발관,이미용,서울특별시 관악구 장군봉2길 11 (봉천동),02-882-5875,264,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 관악구 장군봉2길 11 (봉천동),126.939596,37.482447,새주소,서울특별시 관악구 봉천동 946-15,8782.0,봉천동,관악구,봉천동,서울특별시 관악구 청룡동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1862,현미식당,한식,서울특별시 서울 구로구 구로중앙로 17,28386088,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 서울 구로구 구로중앙로 17,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
1863,현대세탁,세탁,서울특별시 금천구 범안로17길 23 (독산동),02-804-1477,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 금천구 범안로17길 23 (독산동),126.894000,37.467595,새주소,서울특별시 금천구 독산동 331-4,8586.0,독산동,금천구,독산동,서울특별시 금천구 독산1동
1864,서울밥집 봉천점,한식,서울특별시 관악구 봉천동 927-18,없음,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 관악구 봉천동 927-18,126.942591,37.481762,구주소,서울특별시 관악구 봉천동 927-18,8784.0,봉천동,관악구,봉천동,서울특별시 관악구 청룡동
1865,은혜세탁,세탁,"서울특별시 금천구 벚꽃로 73, 현대종합상가 103호 (독산동, 금천현대아파트)",02-892-1230,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,"서울특별시 금천구 벚꽃로 73, 현대종합상가 103호 (독산동, 금천현대아파트)",126.890426,37.462326,새주소,서울특별시 금천구 독산동 711-2,8596.0,독산동,금천구,독산동,서울특별시 금천구 독산1동


In [ ]:
# 구가 잘 매칭되었는 지 확인 -> nan값 존재 -> 결측치 확인
price['구'].unique()

array(['중랑구', '관악구', nan, '송파구', '강서구', '양천구', '구로구', '광진구', '중구', '동대문구',
       '노원구', '종로구', '용산구', '강동구', '영등포구', '성북구', '마포구', '강북구', '서초구',
       '서대문구', '도봉구', '성동구', '금천구', '은평구', '강남구', '동작구'], dtype=object)

In [ ]:
# 표준구주소가 서울특별시로만 되어 있어 구, 동이 매칭이 안되어 nan값이 출력하는 경우 발생, 업소 주소로 해도, 구가 나오지 않고 도로명 주소(ex.22길)이 나오는 경우가 발생.
price[price['구'].isna()]

,업소명,분류코드명,업소 주소,업소 전화번호,추천수,업소 사진,입력주소,X,Y,주소구분,표준구주소,우편번호,법정동명,구,동,주소
5,헤어디자인하우스,이미용,"서울특별시 서울 관악구 은천로 95, 벽산블루밍상가 4층",02-3285-8002,258,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,"서울특별시 서울 관악구 은천로 95, 벽산블루밍상가 4층",126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
11,호계대중사우나,기타서비스업,"서울특별시 서울 중랑구 봉화산로56길 153, 지하2층",02-433-5490,210,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,"서울특별시 서울 중랑구 봉화산로56길 153, 지하2층",126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
17,장미미용실,이미용,서울특별시 서울 중랑구 용마산로129길 61-4,NaN,137,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 서울 중랑구 용마산로129길 61-4,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
58,아주세탁,세탁,서울특별시 서울 송파구 오금로24길 13,02-420-0235,56,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 서울 송파구 오금로24길 13,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
69,동춘원,중식,서울특별시 서울 중랑구 망우로66길 57,02-434-9454,9,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 서울 중랑구 망우로66길 57,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,락궁,중식,서울특별시 서울 노원구 마들로 31,02-949-8585,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 서울 노원구 마들로 31,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
1858,새마을 미용실,이미용,서울특별시 서울 양천구 은행정로5길 48-1,02-2643-6110,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 서울 양천구 은행정로5길 48-1,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
1859,민들레,한식,서울특별시 서울 마포구 잔다리로 132,02-3142-8887,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 서울 마포구 잔다리로 132,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
1860,만나식당,한식,"서울특별시 서울 동대문구 장한로25길 8 (장안동, 1층)",02-2244-4314,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,"서울특별시 서울 동대문구 장한로25길 8 (장안동, 1층)",126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 따라서, 정규 표현식을 사용하여 '구'를 추출해 결측치를 대체.
price['구'] = price['구'].fillna(price['업소 주소'].str.extract(r'(\b[가-힣]+구\b)')[0])

In [ ]:
# 확인 결과, 이외에도 결측치 존재.
price['구'].unique()

array(['중랑구', '관악구', '송파구', '강서구', '양천구', '구로구', '광진구', '중구', '동대문구',
       '노원구', '종로구', '용산구', '강동구', '영등포구', '성북구', '마포구', '강북구', '서초구',
       '서대문구', '도봉구', '성동구', '금천구', nan, '은평구', '강남구', '동작구'],
      dtype=object)

In [ ]:
# 결측치 파악해본 결과, 입력주소, 업소 주소, 표준구주소에 '구'가 없음을 발견. 따라서 drop함.
price[price['구'].isna()]

,업소명,분류코드명,업소 주소,업소 전화번호,추천수,업소 사진,입력주소,X,Y,주소구분,표준구주소,우편번호,법정동명,구,동,주소
140,소문난맛함흥냉면,한식,서울특별시 무학봉16길 1 (하왕십리동),02-2299-5537,5,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 무학봉16길 1 (하왕십리동),126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
199,지앤미미용실,이미용,서울특별시 화랑로19길 81,02-941-0980,4,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 화랑로19길 81,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
437,포토베베스튜디오,기타서비스업,서울특별시 도림로12길 13 (구로동),02-862-1050,2,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 도림로12길 13 (구로동),126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
479,개성시대,이미용,서울특별시 풍성로 6길 9,02-477-2939,2,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 풍성로 6길 9,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
626,송하정,한식,서울특별시 무학봉길 77 (하왕십리동),02-2291-3346,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 무학봉길 77 (하왕십리동),126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
666,조선부뚜막,한식,서울특별시 동소문로6길 4-23,02-762-8850,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 동소문로6길 4-23,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
675,맨투맨,이미용,서울특별시 금호산길 56-1,02-2282-3043,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 금호산길 56-1,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
699,이가네김밥마을,한식,서울특별시 (일원동)양재대로33길 21,02-3412-2692,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 (일원동)양재대로33길 21,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
700,正닭곰탕,한식,서울특별시 (도곡동)언주로30길 13 대림아크로텔 지하 11호,02-2187-6358,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 (도곡동)언주로30길 13 대림아크로텔 지하 11호,126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN
714,바다장어초이408,한식,서울특별시 독서당로 295-5(금호동3가),0507-1311-1733,0,http://sftc.seoul.go.kr/mulga/inc/img_view.jsp...,서울특별시 독서당로 295-5(금호동3가),126.977878,37.566411,판별불가,서울특별시,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 구가 없는 열은 drop
price.dropna(subset=['구'], inplace=True)

In [ ]:
price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1824 entries, 0 to 1866
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   업소명      1824 non-null   object 
 1   분류코드명    1824 non-null   object 
 2   업소 주소    1824 non-null   object 
 3   업소 전화번호  1711 non-null   object 
 4   추천수      1824 non-null   int64  
 5   업소 사진    1824 non-null   object 
 6   입력주소     1824 non-null   object 
 7   X        1824 non-null   float64
 8   Y        1824 non-null   float64
 9   주소구분     1824 non-null   object 
 10  표준구주소    1824 non-null   object 
 11  우편번호     1112 non-null   float64
 12  법정동명     1112 non-null   object 
 13  구        1824 non-null   object 
 14  동        1118 non-null   object 
 15  주소       1112 non-null   object 
dtypes: float64(3), int64(1), object(12)
memory usage: 242.2+ KB


In [ ]:
# 착한가게업소 데이터셋 저장
price.to_excel('price_geo.xlsx')